# Install required libraries

In [1]:
!pip install transformers datasets huggingface_hub

In [2]:
from huggingface_hub import login

login(token='hf_xxx')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/kevin/.cache/huggingface/token
Login successful


# Explore the dataset

In [1]:
dataset_name = "nanyang-technological-university-singapore/hkcancor"
# dataset_name = "AlienKevin/cc100-yue-tagged"

# model_name = "AlienKevin/bert_base_cantonese_pos_hkcancor"
# checkpoint = "indiejoseph/bert-base-cantonese"

model_name = "AlienKevin/electra_hongkongese_small_pos_hkcancor"
checkpoint = "toastynews/electra-hongkongese-small-discriminator"

max_length = 512
num_train_epochs = 3

## Load dataset

For this task, we'll be using the `jnlpba` [dataset](https://huggingface.co/datasets/jnlpba).

In [2]:
from datasets import load_dataset

dataset = load_dataset(dataset_name, trust_remote_code=True)

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'speaker', 'turn_number', 'tokens', 'transcriptions', 'pos_tags_prf', 'pos_tags_ud'],
        num_rows: 10801
    })
})

In [4]:
label_names = dataset["train"].features["pos_tags_ud"].feature.names

label_names

['V',
 'NOUN',
 'ADP',
 'AUX',
 'VERB',
 'X',
 'PRON',
 'PROPN',
 'PART',
 'PUNCT',
 'CCONJ',
 'ADV',
 'NUM',
 'INTJ',
 'DET',
 'ADJ']

In [5]:
example = dataset["train"][0]

example

{'conversation_id': 'TN001-DR300497-WAI3C',
 'speaker': 'A',
 'turn_number': 0,
 'tokens': ['喂',
  '遲',
  '啲',
  '去',
  '唔',
  '去',
  '旅行',
  '啊',
  '？',
  '你',
  '老公',
  '有冇',
  '平',
  '機票',
  '啊',
  '？'],
 'transcriptions': ['wai3',
  'ci4',
  'di1',
  'heoi3',
  'm4',
  'heoi3',
  'leoi5hang4',
  'aa3',
  'VQ6',
  'nei5',
  'lou5gung1',
  'jau5mou5',
  'peng4',
  'gei1piu3',
  'aa3',
  'VQ6'],
 'pos_tags_prf': [24,
  9,
  72,
  75,
  21,
  75,
  80,
  116,
  83,
  64,
  50,
  76,
  9,
  50,
  116,
  83],
 'pos_tags_ud': [13, 15, 8, 4, 11, 4, 1, 8, 9, 6, 1, 4, 15, 1, 8, 9]}

The `tokens` are the words in the sentence, and the `pos_tags_ud` are the corresponding labels.

In [6]:
print("Token => Label Name\n")

pre_length = 15
for token, tag in zip(example["tokens"], example["pos_tags_ud"]):
  tag_label = label_names[tag]
  string = token
  while len(string) != pre_length:
    string += " "
  print(f"{string} => {tag_label}")

Token => Label Name

喂               => INTJ
遲               => ADJ
啲               => PART
去               => VERB
唔               => ADV
去               => VERB
旅行              => NOUN
啊               => PART
？               => PUNCT
你               => PRON
老公              => NOUN
有冇              => VERB
平               => ADJ
機票              => NOUN
啊               => PART
？               => PUNCT


In [7]:
import random

# Set seed for reproducibility
random.seed(42)

# Separate 100 random rows for a new validation split
train_dataset = dataset["train"]
validation_indices = random.sample(range(len(train_dataset)), 100)
validation_dataset = train_dataset.select(validation_indices)
train_dataset = train_dataset.select([i for i in range(len(train_dataset)) if i not in validation_indices])

# Update the dataset dictionary
dataset["train"] = train_dataset
dataset["validation"] = validation_dataset

# Display the new dataset splits
dataset

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'speaker', 'turn_number', 'tokens', 'transcriptions', 'pos_tags_prf', 'pos_tags_ud'],
        num_rows: 10701
    })
    validation: Dataset({
        features: ['conversation_id', 'speaker', 'turn_number', 'tokens', 'transcriptions', 'pos_tags_prf', 'pos_tags_ud'],
        num_rows: 100
    })
})

## Tokenization

The Tokenizer is used to convert sentences into [sub-words](https://towardsdatascience.com/a-comprehensive-guide-to-subword-tokenisers-4bbd3bad9a7c).

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Let's tokenize our example.

In [9]:
example

{'conversation_id': 'TN001-DR300497-WAI3C',
 'speaker': 'A',
 'turn_number': 0,
 'tokens': ['喂',
  '遲',
  '啲',
  '去',
  '唔',
  '去',
  '旅行',
  '啊',
  '？',
  '你',
  '老公',
  '有冇',
  '平',
  '機票',
  '啊',
  '？'],
 'transcriptions': ['wai3',
  'ci4',
  'di1',
  'heoi3',
  'm4',
  'heoi3',
  'leoi5hang4',
  'aa3',
  'VQ6',
  'nei5',
  'lou5gung1',
  'jau5mou5',
  'peng4',
  'gei1piu3',
  'aa3',
  'VQ6'],
 'pos_tags_prf': [24,
  9,
  72,
  75,
  21,
  75,
  80,
  116,
  83,
  64,
  50,
  76,
  9,
  50,
  116,
  83],
 'pos_tags_ud': [13, 15, 8, 4, 11, 4, 1, 8, 9, 6, 1, 4, 15, 1, 8, 9]}

In [10]:
inputs = tokenizer(
    example["tokens"],
    is_split_into_words=True,
)

inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [11]:
print(inputs["input_ids"])

[2, 9370, 21175, 9354, 8964, 9273, 8964, 12679, 19484, 9321, 24244, 8058, 17855, 8482, 12936, 8505, 11114, 13596, 16527, 9321, 24244, 3]


Since the tokenizer converts the words into sub-words, the number of tokens will be greater than the number of labels, because each word has been split into one or more sub-words.

In [12]:
len(inputs["input_ids"])

22

In [13]:
len(example["pos_tags_ud"])

16

Now, let's create a function which takes in a group of examples, tokenize each example, and aligns their labels.

In [14]:
# https://huggingface.co/docs/transformers/en/tasks/token_classification#preprocess
def tokenize_and_align(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=max_length)

    labels = []
    for i, label in enumerate(examples[f"pos_tags_ud"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Let's try this function on a group of examples.

In [15]:
examples = dataset["train"][:5]

In [16]:
inputs = tokenize_and_align(examples)

In [17]:
for ex_input_ids, ex_labels in zip(inputs["input_ids"], inputs["labels"]):
  print(f">>> Length of input_ids: {len(ex_input_ids)}")
  print(f">>> Length of labels: {len(ex_labels)}")
  print()

>>> Length of input_ids: 22
>>> Length of labels: 22

>>> Length of input_ids: 20
>>> Length of labels: 20

>>> Length of input_ids: 7
>>> Length of labels: 7

>>> Length of input_ids: 4
>>> Length of labels: 4

>>> Length of input_ids: 19
>>> Length of labels: 19



Nice! We have tokenized the sentences, and made sure that the labels for each sentence are the same length.

Notice how each separate example, however, is of different length. That's because we haven't applied **padding** yet, however we will take care of this later.

In [18]:
tokenized_datasets = dataset.map(
    tokenize_and_align,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10701
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [20]:
print(tokenized_datasets["train"][0]['input_ids'])
print(tokenized_datasets["train"][0]['labels'])

[2, 9370, 21175, 9354, 8964, 9273, 8964, 12679, 19484, 9321, 24244, 8058, 17855, 8482, 12936, 8505, 11114, 13596, 16527, 9321, 24244, 3]
[-100, 13, 15, 8, 4, 11, 4, 1, -100, 8, 9, 6, 1, -100, 4, -100, 15, 1, -100, 8, 9, -100]


## Data Collation

Data Collation means taking our dataset and organanizing it in mini-batches.

You may have noticed that we haven't padded our dataset yet, as models require each tensor to be of equal length. Padding the entire dataset at once would be inefficient, as we would be padding each tensor to the length of the longest tensor in the dataset.

Instead, we can do this for each mini-batch, so each tensor is only padded up to the largest tensor in its mini-batch. This saves unnecessary RAM and computation.

The data collator takes care of this for us.

In [21]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Let's test out our data collator on a small batch.

In [22]:
batch_pre_collation = [ tokenized_datasets["train"][i] for i in range(5) ]

In [23]:
for example in batch_pre_collation:
  print(f">>> Length: {len(example['input_ids'])}")

>>> Length: 22
>>> Length: 20
>>> Length: 7
>>> Length: 4
>>> Length: 19


Right now, each example has a different length.

After we apply padding through the data collator, each example will have the length of the longest example in the mini-batch.

In [24]:
batch_collated = data_collator(batch_pre_collation)

In [25]:
for example in batch_collated["input_ids"]:
  print(f">>> Length: {len(example)}")

>>> Length: 22
>>> Length: 22
>>> Length: 22
>>> Length: 22
>>> Length: 22


In [26]:
batch_collated["input_ids"].shape

torch.Size([5, 22])

In [27]:
batch_collated["labels"].shape

torch.Size([5, 22])

## Prepare dataset for fine-tuning

Now that we've set up our data collator, let's apply it to our entire dataset.

# Fine-tuning!

## Load `bert-base-cantonese` model

Before loading the model, let's create some maps which will be used to go back and forth between ids and labels.

In [28]:
id2label = { i:k for i, k in enumerate(label_names) }
label2id = { v:k for k, v in id2label.items() }

In [29]:
id2label

{0: 'V',
 1: 'NOUN',
 2: 'ADP',
 3: 'AUX',
 4: 'VERB',
 5: 'X',
 6: 'PRON',
 7: 'PROPN',
 8: 'PART',
 9: 'PUNCT',
 10: 'CCONJ',
 11: 'ADV',
 12: 'NUM',
 13: 'INTJ',
 14: 'DET',
 15: 'ADJ'}

In [30]:
label2id

{'V': 0,
 'NOUN': 1,
 'ADP': 2,
 'AUX': 3,
 'VERB': 4,
 'X': 5,
 'PRON': 6,
 'PROPN': 7,
 'PART': 8,
 'PUNCT': 9,
 'CCONJ': 10,
 'ADV': 11,
 'NUM': 12,
 'INTJ': 13,
 'DET': 14,
 'ADJ': 15}

Now let's load our model.

The reason we created those maps above is so that the Inference API on the Hugging Face website can show us the label names such as "DNA" and "protein" instead of the ids.

In [31]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

if "electra" in model_name:
    # Make the model's parameters contiguous
    for param in model.parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at toastynews/electra-hongkongese-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model knows how many classes there are because of the maps we provided earlier.

In [32]:
model.config.num_labels

16

In [33]:
assert model.config.num_labels == len(label_names)

## Prepare model

## PushToHubCallback

This callbacks tells HuggingFace to push the model to your HuggingFace profile while the model is training.

To make this work, ensure that you have connected your HuggingFace account to the notebook.

## Train!

In [34]:
import wandb
from transformers import Trainer, TrainingArguments

wandb.init(project="CantoBERT")

training_args = TrainingArguments(
    output_dir=f'./{model_name.split("/")[-1]}',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=0,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="epoch",
    push_to_hub=False,
    report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
wandb.finish()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenize

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/2007 [00:00<?, ?it/s]

{'loss': 2.7471, 'grad_norm': 1.6966887712478638, 'learning_rate': 1.9900348779272545e-05, 'epoch': 0.01}
{'loss': 2.6772, 'grad_norm': 1.7285258769989014, 'learning_rate': 1.9800697558545096e-05, 'epoch': 0.03}
{'loss': 2.5853, 'grad_norm': 1.8951977491378784, 'learning_rate': 1.970104633781764e-05, 'epoch': 0.04}
{'loss': 2.5092, 'grad_norm': 1.7884441614151, 'learning_rate': 1.9601395117090184e-05, 'epoch': 0.06}
{'loss': 2.4149, 'grad_norm': 1.8379368782043457, 'learning_rate': 1.9501743896362734e-05, 'epoch': 0.07}
{'loss': 2.3312, 'grad_norm': 1.7341512441635132, 'learning_rate': 1.9402092675635278e-05, 'epoch': 0.09}
{'loss': 2.2581, 'grad_norm': 1.8665177822113037, 'learning_rate': 1.9302441454907822e-05, 'epoch': 0.1}
{'loss': 2.1931, 'grad_norm': 1.6170521974563599, 'learning_rate': 1.920279023418037e-05, 'epoch': 0.12}
{'loss': 2.1127, 'grad_norm': 1.9911563396453857, 'learning_rate': 1.9103139013452916e-05, 'epoch': 0.13}
{'loss': 2.0437, 'grad_norm': 1.6713836193084717, 'l

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.29497379064559937, 'eval_runtime': 0.5101, 'eval_samples_per_second': 196.033, 'eval_steps_per_second': 13.722, 'epoch': 2.4}
{'loss': 0.3568, 'grad_norm': 1.3276641368865967, 'learning_rate': 3.956153462879921e-06, 'epoch': 2.41}
{'loss': 0.3215, 'grad_norm': 1.35435950756073, 'learning_rate': 3.8565022421524665e-06, 'epoch': 2.42}
{'loss': 0.3241, 'grad_norm': 1.1545326709747314, 'learning_rate': 3.7568510214250124e-06, 'epoch': 2.44}
{'loss': 0.332, 'grad_norm': 1.1092991828918457, 'learning_rate': 3.657199800697559e-06, 'epoch': 2.45}
{'loss': 0.3316, 'grad_norm': 1.5443695783615112, 'learning_rate': 3.5575485799701047e-06, 'epoch': 2.47}
{'loss': 0.2928, 'grad_norm': 0.925728976726532, 'learning_rate': 3.4578973592426514e-06, 'epoch': 2.48}
{'loss': 0.338, 'grad_norm': 1.109576940536499, 'learning_rate': 3.358246138515197e-06, 'epoch': 2.5}
{'loss': 0.3192, 'grad_norm': 1.0123553276062012, 'learning_rate': 3.258594917787743e-06, 'epoch': 2.51}
{'loss': 0.3136, 'gra

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.28457602858543396, 'eval_runtime': 0.5153, 'eval_samples_per_second': 194.066, 'eval_steps_per_second': 13.585, 'epoch': 2.7}
{'loss': 0.3014, 'grad_norm': 0.8153993487358093, 'learning_rate': 1.963129048330842e-06, 'epoch': 2.71}
{'loss': 0.3021, 'grad_norm': 1.1058731079101562, 'learning_rate': 1.8634778276033883e-06, 'epoch': 2.72}
{'loss': 0.3103, 'grad_norm': 2.494328022003174, 'learning_rate': 1.7638266068759345e-06, 'epoch': 2.74}
{'loss': 0.3247, 'grad_norm': 1.0259900093078613, 'learning_rate': 1.6641753861484806e-06, 'epoch': 2.75}
{'loss': 0.2986, 'grad_norm': 0.9228556156158447, 'learning_rate': 1.5645241654210267e-06, 'epoch': 2.77}
{'loss': 0.3221, 'grad_norm': 1.3900444507598877, 'learning_rate': 1.4648729446935724e-06, 'epoch': 2.78}
{'loss': 0.2941, 'grad_norm': 1.0223788022994995, 'learning_rate': 1.3652217239661186e-06, 'epoch': 2.8}
{'loss': 0.3094, 'grad_norm': 1.8018395900726318, 'learning_rate': 1.2655705032386647e-06, 'epoch': 2.81}
{'loss': 0.33

eval/loss,█▄▃▂▂▁▁▁▁
eval/runtime,█▂▂▁▂▁▂▁▁
eval/samples_per_second,▁▆▆▇▆▇▆██
eval/steps_per_second,▁▆▆▇▆▇▆██
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▅▅▅▅▄▅▃▄▄▄▃▃▂▄▂▆▆▃▅▃▄▂▂▂▁▃▆▁▂▃▂▁▁▁█▃▁▅▆▃
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▆▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.28458
eval/runtime,0.5153


In [35]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/53.7M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlienKevin/electra_hongkongese_small_pos_hkcancor/commit/7d96dd4113a154dff2ad1cd91d59aca9e8cfb526', commit_message='End of training', commit_description='', oid='7d96dd4113a154dff2ad1cd91d59aca9e8cfb526', pr_url=None, pr_revision=None, pr_num=None)

# Inference

Now that our model is ready, we can try it out using the nifty `pipeline` API.

In [36]:
from transformers import pipeline

classifier = pipeline(
    "token-classification",
    model_name,
    grouped_entities=True,
)

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/53.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/128k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/600k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/kevin/miniconda3/envs/cantobert/lib/python3.12/site-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [37]:
input = "醫院喺邊度？"

In [38]:
classifier(input)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'NOUN',
  'score': 0.83947504,
  'word': '醫 院',
  'start': 0,
  'end': 2},
 {'entity_group': 'ADP',
  'score': 0.40372804,
  'word': '喺',
  'start': 2,
  'end': 3},
 {'entity_group': 'PRON',
  'score': 0.9002198,
  'word': '邊',
  'start': 3,
  'end': 4},
 {'entity_group': 'ADV',
  'score': 0.7424585,
  'word': '度',
  'start': 4,
  'end': 5},
 {'entity_group': 'PUNCT',
  'score': 0.91792315,
  'word': '？',
  'start': 5,
  'end': 6}]

: 